Based on [this tutorial](https://medium.com/geekculture/fine-tune-eleutherai-gpt-neo-to-generate-netflix-movie-descriptions-in-only-47-lines-of-code-40c9b4c32475)

In [1]:
import json
import numpy as np
from typing import List, Tuple

from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset, random_split
import torch

from CounterSpeechGenerator import process_train_data

In [2]:
class HSCSDataset(Dataset):
    def __init__(self, txt_list: List[Tuple[str, str]], target_to_tok: Tuple[dict, dict], tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        
        target_to_hs_tok = target_to_tok[0]
        target_to_cs_tok = target_to_tok[1]
        for (hs, cs, target) in txt_list:
            hs_tok = target_to_hs_tok[target]
            cs_tok = target_to_cs_tok[target]
            encodings_dict = tokenizer(hs_tok + hs + cs_tok + cs + eos_tok, truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [2]:
gpt2_model_name = 'gpt2-medium'
out_dir = './Results'
save_name = 'testmodel'    
epochs = 1

In [3]:
# process data
hs_cs_pairs, targets = process_train_data()

In [4]:
# create tokens for each target type
target_types = np.unique(targets)
special_toks_hs = []
special_toks_cs = []
for target in target_types:
    special_toks_hs.append('<|' + target + '_HS|>')
    special_toks_cs.append('<|' + target + '_CS|>')
special_toks = special_toks_hs + special_toks_cs
eos_tok = '<|endoftext|>'
target_to_hs_tok = {target: tok for (target, tok) in zip(target_types, special_toks_hs)}
target_to_cs_tok = {target: tok for (target, tok) in zip(target_types, special_toks_cs)}

### Train Model

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained(gpt2_model_name, eos_token=eos_tok, pad_token='<|pad|>')
tokenizer.add_tokens(special_toks)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


16

In [11]:
tokenizer.save_pretrained(save_name + '_tokenizer')

('testmodel_tokenizer/tokenizer_config.json',
 'testmodel_tokenizer/special_tokens_map.json',
 'testmodel_tokenizer/vocab.json',
 'testmodel_tokenizer/merges.txt',
 'testmodel_tokenizer/added_tokens.json')

In [7]:
model = GPT2LMHeadModel.from_pretrained(gpt2_model_name)
model.resize_token_embeddings(len(tokenizer))
max_length = max([len(tokenizer.encode(hs+cs)) for (hs, cs, _) in hs_cs_pairs])

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [8]:
# train/val split
dataset = HSCSDataset(hs_cs_pairs, (target_to_hs_tok, target_to_cs_tok), tokenizer, max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [9]:
# train model
training_args = TrainingArguments(output_dir=out_dir, num_train_epochs=epochs, 
                                  weight_decay=0.05, logging_dir='./Logs', report_to = 'none',
                                  save_total_limit=2)
    
Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset, 
        data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                    'attention_mask': torch.stack([f[1] for f in data]),
                                    'labels': torch.stack([f[0] for f in data])}).train()
model.save_pretrained(save_name)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4502
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 563


Step,Training Loss
500,0.836600


Saving model checkpoint to ./Results/checkpoint-500
Configuration saved in ./Results/checkpoint-500/config.json
Model weights saved in ./Results/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in testmodel/config.json
Model weights saved in testmodel/pytorch_model.bin


### Load Model

In [5]:
load_name = 'Models/gpt2_medium'

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained(load_name + '_tokenizer')

In [7]:
model = GPT2LMHeadModel.from_pretrained(load_name)

In [8]:
bad_words = list(target_to_hs_tok.values()) + list(target_to_cs_tok.values())

In [9]:
bad_word_ids = [tokenizer(bad_word).input_ids[0] for bad_word in bad_words]

### Generate Counter Speech

An example from our training data:

In [31]:
# just get an example hate speech from training set
ex_idx = 103
example_hs = hs_cs_pairs[ex_idx][0]
example_cs = hs_cs_pairs[ex_idx][1]
example_target = hs_cs_pairs[ex_idx][2]
print(example_hs)
print(example_cs)
print(example_target)

Women are basically childlike, they remain this way most of their lives. Soft and emotional. It has devastated our once great patriarchal civilizations.
Both men and women can possess characteristics like 'soft' and 'emotional',which are not determined by gender, nor are they a negative thing. Maybe we could all use a bit more compassion and 'emotion' in today's world?
WOMEN


In [32]:
hs_tok = target_to_hs_tok[example_target]
cs_tok = target_to_cs_tok[example_target]
generated = tokenizer(hs_tok + example_hs + cs_tok, return_tensors="pt").input_ids 

In [33]:
sample_outputs = model.generate(generated, do_sample=True, top_k=40, bad_word_ids=bad_word_ids,
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: <|WOMEN_HS|> Women are basically childlike, they remain this way most of their lives. Soft and emotional. It has devastated our once great patriarchal civilizations. <|WOMEN_CS|> Patriarchy is a curse: since they are young, men are told to be tough and strong, to bear all the burdens while being emotionless. Women have been taught to be both emotionally and physically strong, and it has led to higher levels of suicide amongst both men and women. I don't see this as an indicator of a "great civilization".
1: <|WOMEN_HS|> Women are basically childlike, they remain this way most of their lives. Soft and emotional. It has devastated our once great patriarchal civilizations. <|WOMEN_CS|> No. We are all guilty of this in our societies: patriarchy and sexism. But the problem with our society right now, is that we allow our prejudices to rule our heads and minds. We allow these misogynistic monsters to rule our societies. This is why we should all stand up against these evil feminist monste

Running on our reddit/tumblr datasets:

In [10]:
from CounterSpeechGenerator import generate_counterspeech

In [11]:
reddit_data_path = 'Data/reddit_hate.json'

In [12]:
tumblr_data_path = 'Data/tumblr_hate.json'

In [25]:
with open(reddit_data_path, 'r') as f:
    raw_reddit_data = json.load(f)

In [13]:
with open(tumblr_data_path, 'r') as f:
    raw_tumblr_data = json.load(f)

In [14]:
raw_reddit_data = raw_tumblr_data

Only keep info we need for counter-speech generation:

In [15]:
reddit_hate = []
reddit_targets = []

for post_num in raw_reddit_data:
    text = raw_reddit_data[post_num]['text']
    target = raw_reddit_data[post_num]['target']

    if target == 'None':
        continue

    reddit_hate.append(text)
    reddit_targets.append(target)

In [ ]:
reddit_targets

In [17]:
reddit_counterspeech = generate_counterspeech(reddit_hate, reddit_targets)

Token indices sequence length is longer than the specified maximum sequence length for this model (1966 > 1024). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` t

In [18]:
reddit_results = {}
for i in range(len(reddit_hate)):
    reddit_results[i] = {
        'hatespeech': reddit_hate[i],
        'target': reddit_targets[i],
        'counterspeech': reddit_counterspeech[i]
    }

In [19]:
with open('Data/tumblr_gpt2_counterspeech.json', 'w') as f:
    json.dump(reddit_results, f)